# 0. Import

In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import similarity_metric as sm
import formality_metric as fm

# 1. Load Dataset

In [3]:
dataframe = pd.read_json('./../data/lyrics_dataset.json')
print(dataframe.shape)
dataframe.head()

(36540, 9)


,index,title,lyrics,genre,mungchi,label,chunk_strategy,sampling_strategy,syllable_limit
0,0,J에게,J 스치는 바람에 J 그대 모습 보이면\n난 오늘도 조용히 그댈 그리워하네\nJ 지...,발라드,"[[1, 3, 3, 1, 2, 2, 3], [1, 3, 3, 2, 5], [1, 3...","[[J, 스치는, 바람에, J, 그대, 모습, 보이면], [난, 오늘도, 조용히, ...",word,one,0
1,1,갯바위,나는 나는 갯바위\n당신은 나를 사랑하는 파도\n어느 고운 바람 불던날\n잔잔히 다...,포크/블루스,"[[2, 2, 3], [3, 2, 4, 2], [2, 2, 2, 3], [3, 3]...","[[나는, 나는, 갯바위], [당신은, 나를, 사랑하는, 파도], [어느, 고운, ...",word,one,0
2,2,약속,별이 유난히도 밝은\n오늘 이 시간이 가면\n그대\n떠난다~는 말이\n나를 슬프~게...,발라드,"[[2, 4, 2], [2, 1, 3, 2], [2], [4, 2], [2, 3, ...","[[별이, 유난히도, 밝은], [오늘, 이, 시간이, 가면], [그대], [떠난다~...",word,one,0
3,3,잃어버린 정,당신의 고운 눈매에 할말을 잊었지만은\n냉정히 돌아선 무정한 사람은 눈물을 모르겠지...,성인가요/트로트,"[[3, 2, 3, 3, 5], [3, 3, 3, 3, 3, 5], [3, 5, 3...","[[당신의, 고운, 눈매에, 할말을, 잊었지만은], [냉정히, 돌아선, 무정한, 사...",word,one,0
4,4,물같은 사랑,그대의 모습 멀어져갔네\n쓸쓸한 기억 남기고\n낭만이 지워진 그 길로\n조용히 멀어...,성인가요/트로트,"[[3, 2, 5], [3, 2, 3], [3, 3, 1, 2], [3, 5], [...","[[그대의, 모습, 멀어져갔네], [쓸쓸한, 기억, 남기고], [낭만이, 지워진, ...",word,one,0


In [4]:
df = dataframe.copy()

# 2. Sampling

In [5]:
ex_title_1 = 'J에게'
golden_lyrics_list = df.loc[(df['title']==ex_title_1) & (df['chunk_strategy']=='line') & (df['sampling_strategy']=='total')]['label'].reset_index(drop=True)[0]

ex_title_2 = '갯바위'
golden_mungchies = df.loc[(df['title']==ex_title_2) & (df['chunk_strategy']=='word') & (df['sampling_strategy']=='one')]['mungchi'].reset_index(drop=True)[0]
golden_labels = df.loc[(df['title']==ex_title_2) & (df['chunk_strategy']=='word') & (df['sampling_strategy']=='one')]['label'].reset_index(drop=True)[0]

ex_title_3 = 'J에게'
predicted_mungchies = df.loc[(df['title']==ex_title_3) & (df['chunk_strategy']=='line') & (df['sampling_strategy']=='one')]['mungchi'].reset_index(drop=True)[0]
predicted_labels = df.loc[(df['title']==ex_title_3) & (df['chunk_strategy']=='line') & (df['sampling_strategy']=='one')]['label'].reset_index(drop=True)[0]

print(ex_title_1)
print(golden_lyrics_list)

print(ex_title_2)
print(golden_mungchies)
print(golden_labels)

print(ex_title_3)
print(predicted_mungchies)
print(predicted_labels)

J에게
[['J 스치는 바람에 J 그대 모습 보이면', '난 오늘도 조용히 그댈 그리워하네', 'J 지난밤 꿈속에 J 만났던 모습은', '내 가슴 속깊이 여울져 남아 있네', 'J 아름다운 여름날이 멀리 사라졌다 해도', 'J 나의 사랑은 아직도 변함 없는데', 'j 난 너를 못잊어 J 난 너를 사랑해', 'J 우리가 걸었던 J 추억의 그 길을', '난 이밤도 쓸쓸히 쓸쓸히 걷고 있네', 'J 아름다운 여름날이 멀리 사라졌다 해도', 'J 나의 사랑은 아직도 변함 없는데', 'j 난 너를 못잊어 J 난 너를 사랑해', 'J 우리가 걸었던 J 추억의 그길을', '난 이밤도 쓸쓸히 쓸쓸히 걷고 있네', '쓸쓸히 걷고 있네 -']]
갯바위
[[2, 2, 3], [3, 2, 4, 2], [2, 2, 2, 3], [3, 3], [4, 3, 2, 3], [4, 3, 2, 3], [2, 4, 3, 3], [2, 3, 3, 4], [2, 2, 3], [6], [2, 4, 2], [4, 2], [2, 2, 3], [3, 2, 4, 2], [2, 3, 4], [3, 3], [2, 4, 3, 3], [2, 3, 3, 4], [2, 2, 3], [6], [2, 4, 2], [4, 2], [2, 2, 3], [3, 2, 4, 2], [2, 3, 4], [3, 3], [2, 2, 3], [3, 2, 4, 2], [2, 3, 4], [3, 3], [3, 3]]
[['나는', '나는', '갯바위'], ['당신은', '나를', '사랑하는', '파도'], ['어느', '고운', '바람', '불던날'], ['잔잔히', '다가와'], ['부드러운', '손길로', '나를', '감싸고'], ['향기로운', '입술도', '내게', '주었지'], ['세찬', '비바람에', '내몸이', '패이고'], ['이는', '파도에', '깨끗이', '부서져도'], ['나의', '생은', '당신의'], ['조각품인것을'], ['나는', '당신으로', '인해'], ['아름다운', '것을'], ['나는', '나는', '

In [6]:
golden_lyrics = golden_lyrics_list[0]
golden_mungchi = golden_mungchies[0]
golden_label = golden_labels[0]
predicted_mungchi = predicted_mungchies[1]
predicted_label = predicted_labels[1]

print(ex_title_1)
print(golden_lyrics)

print(ex_title_2)
print(golden_mungchi)
print(golden_label)

print(ex_title_3)
print(predicted_mungchi)
print(predicted_label)

J에게
['J 스치는 바람에 J 그대 모습 보이면', '난 오늘도 조용히 그댈 그리워하네', 'J 지난밤 꿈속에 J 만났던 모습은', '내 가슴 속깊이 여울져 남아 있네', 'J 아름다운 여름날이 멀리 사라졌다 해도', 'J 나의 사랑은 아직도 변함 없는데', 'j 난 너를 못잊어 J 난 너를 사랑해', 'J 우리가 걸었던 J 추억의 그 길을', '난 이밤도 쓸쓸히 쓸쓸히 걷고 있네', 'J 아름다운 여름날이 멀리 사라졌다 해도', 'J 나의 사랑은 아직도 변함 없는데', 'j 난 너를 못잊어 J 난 너를 사랑해', 'J 우리가 걸었던 J 추억의 그길을', '난 이밤도 쓸쓸히 쓸쓸히 걷고 있네', '쓸쓸히 걷고 있네 -']
갯바위
[2, 2, 3]
['나는', '나는', '갯바위']
J에게
[14]
['난 오늘도 조용히 그댈 그리워하네']


# 2. Evaluation

## 2.1 유사성

bert 기반 embedding 모델 -> cosine similarity를 통한 유사도 측정

In [7]:
# Load the pre-trained BERT model and tokenizer
model = AutoModel.from_pretrained("kakaobank/kf-deberta-base")
tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### 2.1.1 의미적 유사도 측정

In [8]:
sm.eval_semantic_sim(model, tokenizer, golden_lyrics, predicted_label)

golden_label_string: J 스치는 바람에 J 그대 모습 보이면 난 오늘도 조용히 그댈 그리워하네 J 지난밤 꿈속에 J 만났던 모습은 내 가슴 속깊이 여울져 남아 있네 J 아름다운 여름날이 멀리 사라졌다 해도 J 나의 사랑은 아직도 변함 없는데 j 난 너를 못잊어 J 난 너를 사랑해 J 우리가 걸었던 J 추억의 그 길을 난 이밤도 쓸쓸히 쓸쓸히 걷고 있네 J 아름다운 여름날이 멀리 사라졌다 해도 J 나의 사랑은 아직도 변함 없는데 j 난 너를 못잊어 J 난 너를 사랑해 J 우리가 걸었던 J 추억의 그길을 난 이밤도 쓸쓸히 쓸쓸히 걷고 있네 쓸쓸히 걷고 있네 -
predicted_label_string: 난 오늘도 조용히 그댈 그리워하네


array([[0.99229074]], dtype=float32)

### 2.1.2 어휘 유사도 측정

In [9]:
corpus = sm.get_full_lyrics_list(df)
inverted_tfidf_dict = sm.get_inverted_tfidf_dictionary(corpus)
sm.eval_lexical_sim(predicted_label, golden_lyrics, inverted_tfidf_dict)

num of songs :  6087


0.728700505638574

## 2.2 형식성

### 2.2.1 Accuracy, MSE

In [12]:
fm.eval_form(golden_mungchi, predicted_mungchi)

(0.0, 52.333333333333336)

### 2.2.2 정확도와 오차제곱평균 그 사이 어딘가
- golden과 predict의 차이가 0일 때   -> 1.0
- golden과 predict의 차이가 1~2일 때 -> 0.5
- golden과 predict의 차이가 3~ 일 때 -> 0.0

In [13]:
fm.eval_ours_form(golden_mungchi, predicted_mungchi)

0.16666666666666666